<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/OCR/Image%20Pre-Processing%20for%20OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>